In [9]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from tensorflow.keras import Model, layers
from tensorflow.keras import Sequential

In [25]:
class NeuralNet(Model):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.encoder = Sequential([
                                   layers.Dense(20, activation='relu'), # Fully connected layer
                                   layers.Dense(30, activation='relu'),
                                   layers.Dense(10),
        ])
        
#     def call(self, x):
#         mu, logvar = tf.split(self.encoder(x), 2, axis=1)
        
#         return mu, logvar


In [26]:
model = NeuralNet()
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

In [10]:
class Encoder(Model):
    def __init__(self, latent_dim):
        super().__init__()
        self.latent_dim = latent_dim
        self.encoder = Sequential([layers.Flatten(), # 1차원 Vector로 만듬
                                   layers.Dense(512, activation='relu'), # Fully connected layer
                                   layers.Dense(256, activation='relu'),
                                   layers.Dense(latent_dim * 2),
        ])
        
    def call(self, x):
        mu, logvar = tf.split(self.encoder(x), 2, axis=1)
        
        return mu, logvar


In [21]:
class Decoder(Model):
    def __init__(self, latent_dim):
        super().__init__()
        self.latent_dim = latent_dim
        self.decoder = Sequential([layers.Input([2, 1])
                                   layers.Dense(256, activation='relu'),
                                   layers.Dense(512, activation='relu'),
                                   layers.Dense(784, activation='sigmoid'),
                                   layers.Reshape((28, 28, 1))
                                  ])
       
    def call(self, z):
        return self.decoder(z)

In [13]:
n_epochs = 50
latent_dim = 2
learning_rate = 1e-3
log_interval = 10

In [22]:
encoder = Encoder(latent_dim)
decoder = Decoder(latent_dim)
optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)

In [15]:
def sample(mu, logvar):
    sample_ = tf.random.normal(mu.shape) # z~N(0,1) => z~(mu, var) ~ mu + sigma * z
    sigma = tf.exp(0.5*logvar)  # exp(0.5 log sigma^(2)) => sigma
    return mu + sample_ * sigma

In [16]:
def train_step(inputs):
    with tf.GradientTape() as tape:
        mu, logvar = encoder(inputs)
        z = sample(mu, logvar)
        x_recon = decoder(z)
        
        
        # loss
        reconstruction_error = tf.reduce_sum(tf.loss.binary_crossentorpy(inputs, x_recon))
        kl = 0.5 * tf.reduce_sum(tf.exp(logvar) + tf.square(mu)-1.-logvar)
        loss = (kl + reconstruction_error)/inputs.shape[0]
        
    vars_ = encoder.trainable_variables + decoder.trainable_variables
    grads_ = tape.gradient(loss, vars_)
    optimizer.apply_gradients(zip(grads_, vars_))
    
    return loo, reconstruction_error, kl

In [ ]:
dataset = tfds.load('mnist', split='train')
train_data = dataset.map(lambda data:tf.cast(data['image']

for epoch in range(1, n_epochs + 1):
    total_loss, total_recon, total_kl = 0, 0, 0
    for x in train_data:
        loss, recon, kl = train_step(x)
        total_loss += loss * x.shape[0]
        total_recon += recon
        total_kl += kl
        
    if epoch % log_interval == 0:
        print(f"{epoch:3d} iteration: ELBO{total_loss / len(dataset):.2f}",
              f"Recon: {total_recon / len(dataset):.2f}",
              f"KL: {total_kl / len(dataset):.2f}")